<a href="https://colab.research.google.com/github/Stijn-Kamp/secure-data-management/blob/main/Welcome_bij_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mod
!pip install blake3
!pip install py-ecc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00


In [ ]:
import random

def miller_rabin(n, k):

    # Implementation uses the Miller-Rabin Primality Test
    # The optimal number of rounds for this test is 40
    # See http://stackoverflow.com/questions/6325576/how-many-iterations-of-rabin-miller-should-i-use-for-cryptographic-safe-primes
    # for justification

    # If number is even, it's a composite number

    if n == 2:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


In [5]:
from __future__ import annotations
from random import randrange
from math import prod
from mod import Mod
from functools import reduce
from blake3 import blake3
from hashlib import sha3_256
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq
import logging

logging.basicConfig(level=logging.DEBUG)

In [ ]:
class MPECK:
    """
    class used for the MPECK scheme
    """

    def __init__(self):
        self.keycount = 0 # number of keys generated

        # Generate a random prime number of 256 bits using the miller rabin test
        self.n: int = randrange(2**256, 2**257)
        while not miller_rabin(self.n, 5000):
            self.n = randrange(2**256, 2**257)

        # Define the hash functions
        def hash1(x: int):
            h = int(blake3(x.to_bytes(length=32, byteorder="big")).hexdigest(), 16)
            return multiply(G1, h)

        def hash2(x: int):
            h = int(sha3_256(x.to_bytes(length=32, byteorder="big")).hexdigest(), 16)
            return multiply(G1, h)

        self.h1 = hash1
        self.h2 = hash2

    def generate_key(self) -> (int, int):
        x: int = randrange(0, self.n)
        y = multiply(G2, x)
        self.keycount += 1
        return (y, x, self.keycount - 1)

    def add_doc(self, public_keys: [], keywords: [int]):
        s = randrange(0, self.n)
        r = randrange(0, self.n)
        A = multiply(G2, r)
        B = {pk[1]: multiply(pk[0], s) for pk in public_keys}
        C = [add(multiply(self.h1(kw), r), multiply(self.h2(kw), s)) for kw in keywords]
        print(type(C))
        return (A, B, C)

    def trapdoor(self, secret_key: int, query: [(int, int)]):
        t = randrange(0, self.n)
        T1 = multiply(G2, t)
        print(type(T1))
        T2 = multiply(reduce(add, [self.h1(qw[0]) for qw in query]), t)
        T3_exp = int((Mod(secret_key, self.n) ** -1) * t)
        T3 = multiply(reduce(add, [self.h2(qw[0]) for qw in query]), T3_exp)
        return (T1, T2, T3, [qw[1] for qw in query])

    def test(self, public_key, S, T) -> bool:
        T1 = T[0]
        CI = reduce(add, [S[2][i] for i in T[3]])
        print(type(CI))
        A = S[0]
        T2 = T[1]
        B = S[1][public_key]
        T3 = T[2]

        a = pairing(T1, CI)
        b = pairing(A, T2)
        c = pairing(B, T3)

        print(a)
        print(b)
        print(c)
        print(b * c)
        return a == b * c

In [6]:
mpeck = MPECK()
key0 = mpeck.generate_key()
S = mpeck.add_doc([(key0[0], key0[2])], [1])
T = mpeck.trapdoor(key0[1], [(1, 0)])


<class 'list'>
<class 'tuple'>


In [7]:
print(mpeck.test(key0[2], S, T))

<class 'tuple'>
(7288576565198867945707274666575712234944825791533672866064430996922992842511, 11292142668540045810694290078800240452410476332523334009588379520564718668610, 3671208998994615405950091049875469644132277661543013598666953772318128512520, 3369629113946997583075159373956290893995648992642454911029301872107899675133, 19567588485895604386319701971144903568057315666982946139491521093131452343737, 1369743132517143755800997794812831441054103328814594972223096815626623980223, 8999335004077851509444747009083209939651163869414408557235986381138862617726, 21036114497684551529333887756722200044076749004561865405424866156683839192159, 19790227151193049162692415687404881446708228551983187057009128135004094177758, 2645500471463593230754407511075401570664129116219476743846574055725778648538, 6029784303921205218663544239606398471961306030882920203536718808328463992323, 20126219670655231187806149532590351329194589996853135371219715280476077486342)
(13688409361177283174239654556653112071967